# Getting started with Tensorflow
Let's begin by importing the Tensorflow library (select the following cell and press <kbd>Ctrl</kbd> (or <kbd>Shift</kbd>)+<kbd>Enter</kbd>):

In [ ]:
import tensorflow as tf

TensorFlow provides multiple APIs. The lowest level API--TensorFlow Core-- provides you with complete programming control. We recommend TensorFlow Core for machine learning researchers and others who require fine levels of control over their models. The higher level APIs are built on top of TensorFlow Core. These higher level APIs are typically easier to learn and use than TensorFlow Core. In addition, the higher level APIs make repetitive tasks easier and more consistent between different users. A high-level API like tf.contrib.learn helps you manage data sets, estimators, training and inference. Note that a few of the high-level TensorFlow APIs--those whose method names contain contrib-- are still in development. It is possible that some contrib methods will change or become obsolete in subsequent TensorFlow releases.

This guide begins with a tutorial on TensorFlow Core. Later, we demonstrate how to implement the same model in tf.contrib.learn. Knowing TensorFlow Core principles will give you a great mental model of how things are working internally when you use the more compact higher level API.

## Tensors
The central unit of data in TensorFlow is the tensor. A tensor consists of a set of primitive values shaped into an array of any number of dimensions. A tensor's rank is its number of dimensions. Here are some examples of tensors:

In [ ]:
Tr0 = tf.constant(3) # a rank 0 tensor; this is a scalar with shape []
Tr1 = tf.constant([1. ,2., 3.]) # a rank 1 tensor; this is a vector with shape [3]
Tr2 = tf.constant([[1., 2., 3.], [4., 5., 6.]]) # a rank 2 tensor; a matrix with shape [2, 3]
Tr3 = tf.constant([[[1., 2., 3.]], [[7., 8., 9.]]]) # a rank 3 tensor with shape [2, 1, 3]

print(Tr0, Tr1, Tr2, Tr3, sep='\n')

## The computational graph
You might think of TensorFlow Core programs as consisting of two discrete sections:
<ol>
    <li> Building the computational graph.
    <li> Running the computational graph.
</ol>
A computational graph is a series of TensorFlow operations arranged into a graph of nodes. Let's build a simple computational graph. Each node takes zero or more tensors as inputs and produces a tensor as an output. One type of node is a <b>constant</b>. Like all TensorFlow constants, it takes no inputs, and it outputs a value it stores internally. We can create two floating point <code>Tensors</code> node1 and node2 as follows:

In [ ]:
node1 = tf.constant(3.0, tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
print(node1, node2, sep='\n')

Notice that printing the nodes does not output the values <code>3.0</code> and <code>4.0</code> as you might expect. Instead, they are nodes that, when evaluated, would produce <code>3.0</code> and <code>4.0</code>, respectively. To actually evaluate the nodes, we must run the computational graph within a <b>session</b>. A session encapsulates the control and state of the TensorFlow runtime.

The following code creates a Session object and then invokes its <code>run</code> method to run enough of the computational graph to evaluate node1 and node2. By running the computational graph in a session as follows:

In [ ]:
sess = tf.Session()
print(sess.run([node1, node2]))

We can build more complicated computations by combining <code>Tensor</code> nodes with operations (Operations are also nodes.). For example, we can add our two constant nodes and produce a new graph as follows:



In [ ]:
node3 = tf.add(node1, node2)
print("node3: ", node3)
print("sess.run(node3): ",sess.run(node3))

As it stands, this graph is not especially interesting because it always produces a constant result. A graph can be parameterized to accept external inputs, known as <b>placeholders</b>. A <b>placeholder</b> is a promise to provide a value later.

In [ ]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b  # + provides a shortcut for tf.add(a, b)

The preceding three lines are a bit like a function or a <a href='http://www.secnetix.de/olli/Python/lambda_functions.hawk'>lambda</a> in which we define two input parameters (a and b) and then an operation on them. We can evaluate this graph with multiple inputs by using the <code>feed_dict</code> parameter to specify Tensors that provide concrete values to these placeholders:

In [ ]:
print(sess.run(adder_node, feed_dict={a: 3, b:4.5}))
print(sess.run(adder_node, feed_dict={a: [1,3], b: [2, 4]}))

We can make the computational graph more complex by adding another operation. For example,

In [ ]:
add_and_triple = adder_node * 3.
print(sess.run(add_and_triple, {a: 3, b:4.5}))

### Variables
In machine learning we will typically want a model that can take arbitrary inputs, such as the one above. To make the model trainable, we need to be able to modify the graph to get new outputs with the same input. <b>Variables</b> allow us to add trainable parameters to a graph. They are constructed with a type and initial value:

In [ ]:
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b

Constants are initialized when you call <code>tf.constant</code>, and their value can never change. By contrast, variables are not initialized when you call <code>tf.Variable</code>. To initialize all the variables in a TensorFlow program, you must explicitly call a special operation as follows:

In [ ]:
init = tf.global_variables_initializer()
sess.run(init)

It is important to realize <code>init</code> is a handle to the TensorFlow sub-graph that initializes all the global variables. Until we call <code>sess.run</code>, the variables are uninitialized.

Since <code>x</code> is a placeholder, we can evaluate <code>linear_model</code> for several values of <code>x</code> simultaneously as follows:

In [ ]:
print(sess.run(linear_model, {x:[1,2,3,4]}))

We've created a model, but we don't know how good it is yet. To evaluate the model on training data, we need a <code>y</code> placeholder to provide the desired values, and we need to write a loss function.

A loss function measures how far apart the current model is from the provided data. We'll use a standard loss model for linear regression, which sums the squares of the deltas between the current model and the provided data. <code>linear_model - y</code> creates a vector where each element is the corresponding example's error delta. We call <code>tf.square</code> to square that error. Then, we sum all the squared errors to create a single scalar that abstracts the error of all examples using <code>tf.reduce_sum</code>:

In [ ]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
print(sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]}))

We could improve this manually by reassigning the values of <code>W</code> and <code>b</code> to the perfect values of <code>-1</code> and <code>1</code>. A variable is initialized to the value provided to <code>tf.Variable</code> but can be changed using operations like tf.assign. For example, <code>W=-1</code> and <code>b=1</code> are the optimal parameters for our model. We can change <code>W</code> and <code>b</code> accordingly:

In [ ]:
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])
print(sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]}))

We guessed the "perfect" values of <code>W</code> and <code>b</code>, but the whole point of machine learning is to find the correct model parameters automatically. We will show how to accomplish this in the next section.

## tf.train API

A complete discussion of machine learning is out of the scope of this tutorial. However, TensorFlow provides <b>optimizers</b> that slowly change each variable in order to minimize the loss function. The simplest optimizer is <b>gradient descent</b>. It modifies each variable according to the magnitude of the derivative of loss with respect to that variable. In general, computing symbolic derivatives manually is tedious and error-prone. Consequently, TensorFlow can automatically produce derivatives given only a description of the model using the function <code>tf.gradients</code>. For simplicity, optimizers typically do this for you. For example,

In [ ]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [ ]:
sess.run(init) # reset values to incorrect defaults.
for i in range(1000):
    sess.run(train, {x:[1,2,3,4], y:[0,-1,-2,-3]})

print(sess.run([W, b]))

Now we have done actual machine learning! Although doing this simple linear regression doesn't require much TensorFlow core code, more complicated models and methods to feed data into your model necessitate more code. Thus TensorFlow provides higher level abstractions for common patterns, structures, and functionality. We will learn how to use some of these abstractions in the next section.

### Complete program

The completed trainable linear regression model is shown here:

In [ ]:
import numpy as np
import tensorflow as tf

# Model parameters
W = tf.Variable([.3], tf.float32)
b = tf.Variable([-.3], tf.float32)

# Model input and output
x = tf.placeholder(tf.float32)
linear_model = W * x + b
y = tf.placeholder(tf.float32)

# loss
loss = tf.reduce_sum(tf.square(linear_model - y)) # sum of the squares

# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

# training data
x_train = [1,2,3,4]
y_train = [0,-1,-2,-3]

# training loop
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init) # reset values to wrong
for i in range(1000):
    sess.run(train, {x:x_train, y:y_train})

# evaluate training accuracy
curr_W, curr_b, curr_loss  = sess.run([W, b, loss], {x:x_train, y:y_train})
sess.close()

print("W: {0} b: {1} loss: {2}".format(curr_W, curr_b, curr_loss))


### Next steps

Now you should have some knowledge of the basics of TensorFlow. See the other [tutorial]() explaining how to use the FFBP package which simplifies the process of building and feedforward training neural networks.